In [2]:
import requests
from config import DATA_DOWNLOAD_URL
import pandas as pd
import io
import gzip
import numpy as np
from datetime import datetime

In [3]:
df = pd.DataFrame(columns=["DATE", "DEP", "RR", "TN", "TX", "TM"])

In [4]:
for i in range(95):
    url = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_latest-2024-2025_RR-T-Vent.csv.gz"
    response = requests.get(url)

    compressed_data = io.BytesIO(response.content)
    decompressed_data = gzip.GzipFile(fileobj=compressed_data)

    tmp = pd.read_csv(decompressed_data, sep=";")
    tmp["DATE"] = pd.to_datetime(tmp["AAAAMMJJ"], format="%Y%m%d") #string to date
    tmp = tmp[tmp["DATE"] >= "1979-01-01"].reset_index(drop=True) #filter by date
    tmp['DEP'] = i+1
    tmp = tmp[["DATE", "DEP", "RR", "TN", "TX", "TM"]] #keep only, in order : Date, departement code, précipitations, temp min, temp max and moy temp
    df = pd.concat([df, tmp])


C:\Users\alexl\AppData\Local\Temp\ipykernel_7620\3345767027.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, tmp])


In [67]:
#data for current market year
for i in range(95):
    urlTP = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_latest-2024-2025_RR-T-Vent.csv.gz"
    responseTP = requests.get(urlTP)

    urlRH = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_latest-2024-2025_autres-parametres.csv.gz" 
    responseRH = requests.get(urlRH)

    compressedDataTP = io.BytesIO(responseTP.content)
    decompressedDataTP = gzip.GzipFile(fileobj=compressedDataTP)
    compressedDataRH = io.BytesIO(responseRH.content)
    decompressedDataRH = gzip.GzipFile(fileobj=compressedDataRH)

    tmpTP = pd.read_csv(decompressedDataTP, sep=";")
    tmpRH = pd.read_csv(decompressedDataRH, sep=";")
    tmp = pd.merge(tmpTP, tmpRH, on=["NUM_POSTE", "AAAAMMJJ"], how="outer")
    tmp["DATE"] = pd.to_datetime(tmp["AAAAMMJJ"], format="%Y%m%d") #string to date
    tmp = tmp[tmp["DATE"] >= "2024-09-01"].reset_index(drop=True) #filter by date
    tmp['DEP'] = i+1
    tmp = tmp[["DATE", "DEP", "RR", "TN", "TX", "TM", "UM"]] #keep only, in order : Date, departement code, précipitations, temp min, temp max, moy temp and relative humidity
    df = pd.concat([df, tmp])
svpMin = 610.78 * np.exp(df["TN"] / (df["TN"] + 237.3) * 17.2694)
svpMean = 610.78 * np.exp(df["TM"] / (df["TM"] + 237.3) * 17.2694)
svpMax = 610.78 * np.exp(df["TX"] / (df["TX"] + 237.3) * 17.2694)
vpdMin = svpMin * (1 - df["UM"]/100) / 1000 #divided by 1000 to convert to kPa
vpdMean = svpMean * (1 - df["UM"]/100) / 1000
vpdMax = svpMax * (1 - df["UM"]/100) / 1000
df["vpd_min"] = vpdMin
df["vpd_mean"] = vpdMean
df["vpd_max"] = vpdMax

C:\Users\alexl\AppData\Local\Temp\ipykernel_9596\3472737142.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, tmp])


In [5]:
df

,DATE,DEP,RR,TN,TX,TM
0,2024-01-01,1,3.2,1.8,8.8,5.5
1,2024-01-02,1,15.2,5.0,10.3,7.6
2,2024-01-03,1,8.0,7.5,13.4,9.9
3,2024-01-04,1,0.0,2.8,11.5,6.6
4,2024-01-05,1,8.4,1.8,6.0,5.1
...,...,...,...,...,...,...
2647,2025-03-13,95,0.2,1.0,8.6,3.5
2648,2025-03-14,95,0.0,-0.8,7.6,3.6
2649,2025-03-15,95,0.0,-0.3,8.1,3.9
2650,2025-03-16,95,0.0,1.0,10.6,5.2


In [6]:
df = df.reset_index(drop=True)

In [7]:
df.to_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-current.csv")

In [ ]:
df.to_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-1950-2023.csv")

Make a mean file for each month, each departement

In [3]:
df = pd.read_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-1950-2023.csv")
df = df[df["DATE"] >= "2003-01-01"]
df = df.dropna()
df = df.drop(columns=["Unnamed: 0"])

In [4]:
dailyDepMean = df.groupby(["DEP", "DATE"]).mean().reset_index()
dailyDepMean['YEAR-MONTH'] = pd.to_datetime(dailyDepMean["DATE"]).dt.to_period('M') #new column with YYYY-MM format

In [5]:
yearlyPrecip = dailyDepMean.groupby(["DEP", "YEAR-MONTH"]).sum()["RR"].reset_index()
yearlyPrecip['MONTH'] = yearlyPrecip['YEAR-MONTH'].dt.month
monthlyPrecip = yearlyPrecip.groupby(["MONTH", "DEP"]).mean()['RR'].reset_index()

In [6]:
yearlyTemp = dailyDepMean[["DEP", "YEAR-MONTH", "TN", "TX", "TM"]].groupby(["DEP", "YEAR-MONTH"]).mean()[["TN", "TX", "TM"]].reset_index()
yearlyTemp['MONTH'] = yearlyTemp['YEAR-MONTH'].dt.month
monthlyTemp = yearlyTemp.groupby(["MONTH", "DEP"]).mean()[["TN", "TX", "TM"]].reset_index()

In [7]:
meteoFranceMean = pd.merge(monthlyPrecip, monthlyTemp, on=["MONTH", "DEP"], how='inner')
meteoFranceMean

,MONTH,DEP,RR,TN,TX,TM
0,1,1,111.142766,-1.012066,5.468150,1.859154
1,1,2,63.543750,1.204320,6.098548,3.519510
2,1,3,60.424884,0.512403,6.722931,3.370427
3,1,4,49.107585,-3.260030,8.497296,1.471727
4,1,5,53.761693,-4.779385,6.117791,-0.398876
...,...,...,...,...,...,...
1135,12,91,62.340454,2.318371,7.686998,4.862220
1136,12,92,46.580000,4.584866,9.176586,6.604274
1137,12,93,44.542857,4.257143,8.658894,6.340369
1138,12,94,64.388095,3.248771,8.242857,5.538479


In [8]:
meteoFranceMean.to_csv("../YieldModel/data/forecast/currentYear/temp_precip_mean_historical.csv")

Make the file for current year

In [86]:
df = pd.read_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-current.csv")
df = df[pd.to_datetime(df["DATE"]) < datetime.today().replace(day=1, hour=0, minute=0, second=0, microsecond=0)] #remove data from current month
df = df.dropna()
df = df.drop(columns=["Unnamed: 0"])

In [88]:
dailyDepMean = df.groupby(["DEP", "DATE"]).mean().reset_index()
dailyDepMean['YEAR-MONTH'] = pd.to_datetime(dailyDepMean["DATE"]).dt.to_period('M') #new column with YYYY-MM format

In [89]:
yearlyPrecip = dailyDepMean.groupby(["DEP", "YEAR-MONTH"]).sum()["RR"].reset_index()
yearlyPrecip['MONTH'] = yearlyPrecip['YEAR-MONTH'].dt.month
monthlyPrecip = yearlyPrecip.groupby(["MONTH", "DEP"]).mean()['RR'].reset_index()

In [90]:
yearlyTemp = dailyDepMean[["DEP", "YEAR-MONTH", "TN", "TX", "TM", "vpd_min", "vpd_max", "vpd_mean"]].groupby(["DEP", "YEAR-MONTH"]).mean()[["TN", "TX", "TM", "vpd_min", "vpd_max", "vpd_mean"]].reset_index()
yearlyTemp['MONTH'] = yearlyTemp['YEAR-MONTH'].dt.month
monthlyTemp = yearlyTemp.groupby(["MONTH", "DEP"]).mean()[["TN", "TX", "TM", "vpd_min", "vpd_max", "vpd_mean"]].reset_index()

In [91]:
currentMeteoFranceMean = pd.merge(monthlyPrecip, monthlyTemp, on=["MONTH", "DEP"], how='inner')
currentMeteoFranceMean

,MONTH,DEP,RR,TN,TX,TM,vpd_min,vpd_max,vpd_mean
0,1,1,175.760000,-0.256129,7.592903,3.376129,0.105159,0.195878,0.142369
1,1,2,123.192857,0.885945,6.060753,3.212289,0.059307,0.087267,0.070670
2,1,3,68.440000,0.220000,8.271613,4.061290,0.126657,0.229078,0.171554
3,1,4,76.387500,-0.887097,10.548790,3.939113,0.124916,0.285799,0.177396
4,1,5,105.975000,-2.553495,7.302419,1.632527,0.122693,0.262170,0.169138
...,...,...,...,...,...,...,...,...,...
547,12,88,104.000000,-0.133692,5.143728,2.260573,0.054734,0.086540,0.067828
548,12,89,52.033333,2.449462,7.269892,4.776344,0.062145,0.088011,0.074244
549,12,90,100.500000,0.248387,5.509677,2.548387,0.058420,0.086967,0.069934
550,12,91,41.166667,3.551613,8.012903,5.716129,0.084960,0.113940,0.097801


In [93]:
currentMeteoFranceMean.to_csv("../YieldModel/data/forecast/currentYear/temp_precip_vpd_mean_current.csv")